<a href="https://colab.research.google.com/github/vectice/vectice-examples/blob/master/Notebooks/Vanilla/Tutorial/Tutorial_Modelling_All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparation

Welcome to the Vectice tutorial notebook!


Through this notebook, we will be illustrating how to log the following information into Vectice using the Vectice Python library:
- Dataset versions
- Model versions
- Runs and lineage

For more information on the tutorial, please refer to the "Vectice Tutorial Page" inside the app.



### Install Vectice and GCS packages

In [ ]:
## Requirements
!pip3 install -q fsspec
!pip3 install -q gcsfs

#Install Vectice Python library 
# In this tutorial we will do code versioning using github, we also support gitlab
# and bitbucket: !pip install -q "vectice[github, gitlab, bitbucket]"
!pip install vectice[github]

In [ ]:
#Verify if Vectice python library was installed
!pip3 show vectice

### Retrieve the data from GCS

We are going to load data stored in Google Cloud Storage, that is provided by Vectice for this tutorial.


In [ ]:
# Download the "JSON file" from the "Vectice Tutorial Page" in the application so that 
# you can access the GCS bucket. The name of the JSON file should be "readerKey.json"

from google.colab import files
uploaded = files.upload()

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

# Double check the json file name below so that it matches the name of the file that you uploaded.
# Note that the key provided for this tutorial does not have permissions for you to write to GCS. 
# You can only use it to read the data.
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'readerKey.json'

# Once your file is loaded you can view your dataset in a Pandas dataframe.
df = pd.read_csv('gs://vectice_tutorial/kc_house_data_cleaned.csv')

# Run head to make sure the data was loaded properly
df.head()


## Connect to your Vectice account

In [ ]:
#Import the Vectice library
from vectice import Vectice
from vectice.models import JobType
from vectice.entity.model import ModelType
import logging
logging.basicConfig(level=logging.INFO)

# Specify the API endpoint for Vectice.
os.environ['VECTICE_API_ENDPOINT']= "beta.vectice.com"

# To use the Vectice Python library, you first need to authenticate your account using an API key.
# You can generate an API key from the Vectice UI, by going to the "My API Keys" section under your profile's picture
# Copy and paste your API key here
os.environ['VECTICE_API_TOKEN'] = "3DE0gaovk.029YNXM5rPxBl7jQzeGp83DE0gaovkRbA14yWKwmdVa6JgqLO0"

# Next, you need to specify the tutorial project where you will run this notebook using a 
# "Project Token". You can find the "Project Token" under the "Settings" tab of your project.

# Copy and paste your Project Token here
# autocode = True enables you to track your git changes for your code automatically every time you execute a run (see below).
vectice = Vectice(project_token="YMwbovaNSaMYqA9ZyR9V", autocode= True)


We will use the scikit-learn library for modeling and tracking experiments directly through the Vectice Python library.

In [ ]:
import string
from math import sqrt

# Load scikit-learn packages
from sklearn.model_selection import train_test_split  # Model Selection
from sklearn.metrics import mean_absolute_error, mean_squared_error  # Model Evaluation
from sklearn.linear_model import LinearRegression  # Linear Regression
from sklearn.tree import DecisionTreeRegressor, plot_tree  # Decision Tree Regression
from sklearn.ensemble import RandomForestRegressor  # Random Forest Regression


### Split dataset into training and testing

Let's split the dataset into train and test sets and save them in GCS.
(The GCS code has been commented out as the data has already been generated). For this section, we will re-use some datasets that have been already created to illustrate dataset versioning.


We can get the list of the datasets existing in our project by calling **vectice.list_datasets()**

In [ ]:
vectice.list_datasets().list

In [ ]:
# The Vectice library automatically detects if there have been changes to the dataset you are using.
# If it detects changes, it will generate a new version of your dataset automatically. 
# For this tutorial, we changed the data since Albert left the company.
# So, the Vectice Python library will create a new dataset version the first time you execute this code.

input_ds_version = vectice.create_dataset_version().with_parent_name("cleaned_king_county_house_data")


# Because we are using Colab in this tutorial example we are going to declare a reference to the code
# manually. This will be added as a reference to the run we are going to create next.
# If you are using your local environment with GIT installed or JupyterLab etc... the code
# tracking is automated.
input_code = Vectice.create_code_version_with_github_uri("https://github.com/vectice/vectice-examples",
                        "Notebooks/Vanilla/Tutorial/Tutorial_Modelling_All.ipynb") 


# Each execution for a given job is called a run. When creating a run you need to specify:
# 1) a job name (mandatory): Job names are unique for each project
# 2) a job type (optional)
# 3) a run name (optional): The run names are unique for each job
# Job names, job types and run names are useful to group and search runs in the Vectice UI.
# For this run, we will use the job name "80/20 Split", the job type "PREPARATION" and the run name "data-prep"
# You can have multiple runs with the same job name
vectice.create_run("84/16 Split", JobType.PREPARATION)
vectice.start_run(inputs=[input_ds_version,input_code])
# We will use an 80/20 split to prepare the data
test_size = 0.2

# We will set the random seed so we always generate the same split.
random_state = 42

train, test = train_test_split(df, test_size = test_size, random_state = random_state)

# We commented out the code to persist the training and testing test in GCS, 
# because we already generated the data for you.
# The key provided for this tutorial give you read access only to GCS.
# We left the code below for convenience, in case you want to use your own credentials and GCS bucket.
# train.to_csv (r'gs://vectice_tutorial/training_data.csv', index = False, header = True)
# test.to_csv (r'gs://vectice_tutorial/testing_data.csv', index = False, header = True)

# Generate X_train, X_test, y_train, y_test, which we will need for modeling
X = df.drop("price", axis=1).values
y = df["price"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)


# Let's create new training and testing datasets.
# We will use the existing dataset created by Albert, so that we can append new 
# dataset versions to it.

train_ds_version = vectice.create_dataset_version().with_parent_name("King_County_Training")
test_ds_version = vectice.create_dataset_version().with_parent_name("King_County_Testing")

# Attach the output datasets to the run.
# run.add_outputs(outputs=[train_ds_version,test_ds_version])
#vectice.end_run(outputs=[train_ds_version,test_ds_version])
# End the run, and attach the output datasets to it.
vectice.end_run(outputs=[train_ds_version,test_ds_version])

# We can preview one of our generated outputs to make sure that everything was executed properly.
X_train


## Modeling

### Linear regression model

First, we will do a basic Linear Regression and observe the baseline accuracy metrics.

In [ ]:
# Each execution for a given job is called a run, for LR we will only do one run.
# Setting a job's name is mandatory when starting a run 
# and is useful to group and search runs in the Vectice UI.

# Linear regression model training
vectice.create_run("LR-Model1", JobType.TRAINING)
vectice.start_run(inputs=[train_ds_version,test_ds_version,input_code])

lr_rg = LinearRegression()
lr_rg.fit(X_train, y_train)
lr_pred = lr_rg.predict(X_test)

# Evaluate Metrics
MAE = mean_absolute_error(lr_pred, y_test)
RMSE = sqrt(mean_squared_error(lr_pred, y_test))

print("Root Mean Squared Error: ", RMSE)
print("Mean Absolute Error: ", MAE)

# Let's log the model we trained along with its metrics, as a new version 
# of the "Regressor" model in Vectice.
## Here we use with_user_version() to create a new model version. You can provide a version name for your model version.
# An error will be thrown if the given user version already exists and if you don't provide a version name,
# the version name will be "Version"+"the version number".
model_version = vectice.create_model_version().with_parent_name("Regressor").with_algorithm("Linear Regression").with_type(ModelType.REGRESSION).with_metric("RMSE",RMSE).with_metric("MAE",MAE).with_user_version()

#run.add_outputs(outputs=[model_version])
vectice.end_run(outputs=[model_version])


We can update a model's type or description by using **vectice.update_model()**

In [ ]:
vectice.update_model(parent_name="Regressor", model_type=ModelType.REGRESSION, description="Model description")

### Decision tree model

In this section let's use the decision tree algorithm and compare the accuracy to the logistic regression algorithm. We will try different values for the tree_depth. We will log the model parameters and metrics in Vectice.

In [ ]:
# We can do a few runs with different max depth for the tree.
# Just change the value below and re-run this cell.
# The model versions you created will show up in the Vectice UI as new versions 
# of the "Regressor" Model. You can easily compare them from there.
tree_depth = 6

vectice.create_run("DT-Model1", JobType.TRAINING)
vectice.start_run(inputs=[train_ds_version,test_ds_version,input_code])

dtr = DecisionTreeRegressor(max_depth=tree_depth, min_samples_split=50)
dtr.fit(X_train,y_train)
dtr_pred = dtr.predict(X_test) 

data_feature_names = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                    'waterfront', 'view', 'condition', 'grade', 'sqft_above',
                    'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
                    'long', 'sqft_living15', 'sqft_lot15']

# Visualize the Decision Tree Model
plt.figure(figsize=(25, 10))
plot_tree(dtr, feature_names=data_feature_names, filled=True, fontsize=10)
attachment_name = "DecisionTree_"+str(tree_depth)+".png"
plt.savefig(attachment_name)
## We can attach the decision tree plot to the model version by using .with_attachments([Attachments])
MAE = mean_absolute_error(dtr_pred, y_test)
RMSE = sqrt(mean_squared_error(dtr_pred, y_test))

print("Root Mean Squared Error:", RMSE)
print("Mean Absolute Error:", MAE)
# Here we use with_user_version() to create a new model version. You can provide a version name 
# for your model version. An error will be thrown if the given user
# version already exists and if you don't provide a version name, the version name will be 
# "Version"+"the version number".
model_version = vectice.create_model_version().with_user_version().with_parent_name("Regressor").with_algorithm("Decision Tree").with_type(ModelType.REGRESSION).with_property("Tree Depth",str(tree_depth)).with_metric("RMSE",RMSE).with_metric("MAE",MAE).with_attachments([attachment_name])

run.add_outputs(outputs=[model_version])

### Random forest model

Let's use the Random Forest Regression and do some hyper-parameter tuning on it.

In [ ]:
# You can modify the parameters below and execute multiple runs to train 
# different versions of RF model.
nb_trees = 30
min_samples = 30

vectice.create_run(job_name = "RF-Model1", job_type = JobType.TRAINING)

vectice.start_run(inputs=[train_ds_version,test_ds_version,input_code])

rf_regressor = RandomForestRegressor(n_estimators=nb_trees, min_samples_leaf=min_samples)
rf_regressor.fit(X_train, y_train)
rf_regressor.score(X_test, y_test)
rf_regressor_pred = rf_regressor.predict(X_test)

MAE = mean_absolute_error(rf_regressor_pred, y_test)
RMSE = sqrt(mean_squared_error(rf_regressor_pred, y_test))

print("Root Mean Squared Error:", RMSE)
print("Mean Absolute Error:", MAE)

# Here's an alternative version to declare metrics
metrics = [("RMSE",RMSE),
          ("MAE",MAE)]
## Here we use with_user_version() to create a new model version. You can provide a version
## name for your model version. An error will be thrown if the given user
## version already exists and if you don't provide a version name, the version name will be "Version"+"the version number".
model_version = vectice.create_model_version().with_parent_name("Regressor").with_algorithm("Random Forest").with_type(ModelType.REGRESSION).with_property("nb_trees",str(nb_trees)).with_property("min_samples",str(min_samples)).with_metrics(metrics).with_user_version()

run.add_outputs(outputs=[model_version])


We can get the list of the models existing in the project by calling **vectice.list_models()**

In [ ]:
vectice.list_models().list

You can also get all the model versions you created in previous runs, for offline analysis and understanding in more details what's driving the models performance.

In [ ]:
# Replace with your model's id
vectice.list_model_versions_dataframe(model_id = 2513)

We can see that the Random Forest Regressor model gives the lowest error and should be the preferred approach despite the complexity of the algorithm. Let's get the list of features' importance to discuss which variables are influencing the model the most.

In [ ]:
columns = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
           'waterfront', 'view', 'condition', 'grade', 'sqft_above',
           'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
           'long', 'sqft_living15', 'sqft_lot15']

importance = pd.DataFrame({'Importance': rf_regressor.feature_importances_ * 100}, index=columns)
importance.sort_values(by="Importance", axis=0, ascending=True).plot(kind="barh", color="b")
plt.xlabel("Variable Importance")
plt.gca().legend_ = None


Thank you and congratulations! You have succesfully completed the notebook part of the tutorial.

In this notebooks we have illustrated how you can capture your experiments, hyper-parameters, dataset versions and metrics using Vectice Python library. 
You can now leverage Vectice UI for analysis, documentation and to engage a business conversation around the findings.

Vectice enables you to:
1. Make your experiments more reproducible.
2. Track the data and code that is used for each experiment and model versions.
3. Document your projects' progress and collaborate with your team in Vectice's UI.
4. Discover previous work and reuse your team knowledge for new projects.

We are constantly improving the Vectice Python library and the Vectice application. Let us know what improvements you would like to see in the solution and what your favorite features are after completing this tutorial. 

Feel free to explore more and come up with your own ideas on how to best start leveraging Vectice!
